# Day 3 - Spark Introduction

This notebook consists of basic exercises for start using the Spark DataFrame API. In the end you will also try out Spark SQL and GraphFrames.

## The Dataset

The dataset that will be used is rating data from the [MovieLens](https://movielens.org/) website. Find information about the data [here](https://s3-eu-west-1.amazonaws.com/orvarsbucket/ml-latest/README.txt)

## Loading Data From S3

[S3](http://docs.aws.amazon.com/AmazonS3/latest/dev/Welcome.html) is a scalable storage service provided by AWS. Two datasets is uploaded to a bucket and can be fetched by the provided access keys. The smaller one can be used for testing purposes.

An external library called [spark-csv](https://github.com/databricks/spark-csv) can be used to parse csv-files for Spark.

An example is given below with how to load an parse a CSV-file and load it into a Spark DataFrame.

In [ ]:
# Set access keys for the S3 bucket.
sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", accessKeyId)
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", secretAccessKey)

# Paths. Change PATH_DATASET to ml-latest/ to get the larger dataset.
PATH_BUCKET = 's3n://orvarsbucket/'
PATH_DATASET = 'ml-latest-small/'

In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a Spark SQL context which is required to work with the DataFrame API.
sql_context = SQLContext(sc)

# Read ratings.csv
filename = 'ratings.csv'
ratings_schema = StructType([ \
    StructField("userId", StringType(), True), \
    StructField("movieId", StringType(), True), \
    StructField("rating", FloatType(), True), \
    StructField("timestamp", IntegerType(), True), \
])

ratings_df = sql_context.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load(PATH_BUCKET + PATH_DATASET + filename, schema=ratings_schema)
    
print 'Loaded ' + str(ratings_df.count()) + ' entries from ' + filename + '\n'


<font color='red'>**EXERCISE: **</font>Load and parse the remaining datasets containing data about links, movies and tags.

In [ ]:
filename = 'links.csv'

# ...

## The DataFrame API

In Spark, a DataFrame is a distributed collection of data organized into named columns. The DataFrame API can be used to perform various relational operations on these collections. Read more about Spark SQL and the DataFrame API [here](http://spark.apache.org/docs/latest/sql-programming-guide.html).

### Operations

The DataFrames offers various different high-level operations for processing your dataset, as displayed with the  **count**-operation in previous code. For example you have **take**, **show** and **collect** to display content from a DataFrame. All these operation are called actions and are executed immediately (eager), unlike transformation (e.g. map, filter etc.) which are performed lazy.

<font color='red'>**EXERCISE: **</font>Try out the various methods for displaying DataFrame content. Figure out why you should think twice before using the **collect** operation.

*Tips: Use TAB for code completion.*

In [ ]:
ratings_df.take(2)

# ...

### Caching

A very useful feature in Spark is the ability to avoid reading data from disk by caching it into memory. This is performed by using the **cache** operation.

<font color='red'>**EXERCISE: **</font> Change the PATH_DATASET variable and read in the larger dataset. Enable timing for your cell execution by looking at the example below. Compare the result of an action, e.g. **count**, using a dataset read from disk with a dataset cached into memory.

*Tips: Remember that the **cache** operation is performed lazy.*

*Tips 2: Data can be un-cached with the **unpersist** operation.*

In [ ]:
%%time

import time

# Sleep for 5 seconds.
time.sleep(5)

# ...

### Filters and Aggregations

For a  visual guide for filters, aggregations and other types of transformations, check [this](http://nbviewer.jupyter.org/github/jkthompson/pyspark-pictures/blob/master/pyspark-pictures.ipynb) link.

<font color='red'>**EXERCISE: **</font>Count the number of movies related to the Action genre.

*Tips: Remember that the genres column can consist of multiple genres!*

In [ ]:
# ...

<font color='red'>**EXERCISE: **</font>Group the rating data by year and count the number of ratings each year.

*Tips: Use the **withColumn** operation to create a new column in the DataFrame.*

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# ...

### Joining DataFrames

To process data over multiple DataFrames one needs to join them together and the **join** operation can help with that.

<font color='red'>**EXERCISE: **</font>Print the link to the IMDB page of your favourite movie.

*Tips: Link to IMDB movies has the syntax http://www.imdb.com/title/tt + imdbId.*

In [ ]:
# ...

### SQL Queries

Data in Spark can also be inserted into tables and queried with a SQL-like syntax through Spark's SQLContext. When registering a table, one can infer a schema through reflecting an existing DataFrame or programmatically specify it as we did when we read the data the first time. Since the data already exists in DataFrames we can infer the schemas from those.

<font color='red'>**EXERCISE: **</font>Use SQL query syntax and count how many movies that have received the highest rating.

In [ ]:
# registerTempTable creates an in-memory table scoped to the cluster. 
# saveAsTable is another function which instead creates a physical table stored in S3, and can thus be used in other clusters.
ratings_df.registerTempTable("ratings")

# ...

### GraphFrames

[GraphX](http://spark.apache.org/docs/latest/graphx-programming-guide.html) is a new component in Spark used for graphs and graph-parallel computation. GraphX is to RDDs as [GraphFrames](http://graphframes.github.io/index.html) are to DataFrames, so GraphFrames is just another interface to work with graphs in similar way that you have done with the DataFrames.

<font color='red'>**EXERCISE: **</font>Below is the start of a simple example from the homepage. Use the GraphFrames API and do the section about [Motif finding](http://graphframes.github.io/user-guide.html#motif-finding) to get a feeling of how it works.

In [ ]:
from graphframes import *

# Create a Vertex DataFrame with unique ID column "id"
verts = sql_context.createDataFrame([
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
    ("d", "David", 29),
    ("e", "Esther", 32),
    ("f", "Fanny", 36)
], ["id", "name", "age"])

# Create an Edge DataFrame with "src" and "dst" columns
edges = sql_context.createDataFrame([
    ("a", "b", "friend"),
    ("b", "c", "follow"),
    ("c", "b", "follow"),
    ("f", "c", "follow"),
    ("e", "f", "follow"),
    ("e", "d", "follow"),
    ("d", "a", "follow")
], ["src", "dst", "relationship"])

# Create a GraphFrame
graph = GraphFrame(verts, edges)

# Search for pairs of vertices with edges in both directions between them.
motifs = graph.find("(a)-[e1]->(b); (b)-[e2]->(a)")
motifs.show()
